The main purpose of this script is to format the introns properly so that they can be overlapped with the mutation data. 

The secondary purpose is to categorize the introns into constitutive introns and alternative introns. 

In [1]:
# Import relevant packages
import pandas as pd

In [5]:
# These are the relevant chromosomes we are interested in
chrom = range(1,23)
chrom.extend(['X','Y'])
chrom_act = ['chr'+str(i) for i in chrom]

In [6]:
# Read in intron file
introns = pd.read_csv("../data/NCBI_RefSeq_Curated_Introns.bed",sep="\t",header=None)
print introns.shape
introns.head()

(517099, 6)


,0,1,2,3,4,5
0,chr1,67093604,67096251,NR_075077.1_intron_0_0_chr1_67093605_r,0,-
1,chr1,67096321,67103237,NR_075077.1_intron_1_0_chr1_67096322_r,0,-
2,chr1,67103382,67111576,NR_075077.1_intron_2_0_chr1_67103383_r,0,-
3,chr1,67111644,67113613,NR_075077.1_intron_3_0_chr1_67111645_r,0,-
4,chr1,67113756,67115351,NR_075077.1_intron_4_0_chr1_67113757_r,0,-


In [7]:
# We only want the refseq ID and none of the other data in column 3
# We also only want introns that are in relevant chromosomes
# We also only want introns that are in mRNAs and not non-coding RNAs
introndata = {"chrom":introns[0],'start':introns[1],'end':introns[2],'refseq_id':introns[3].str.split(pat='.',expand=True)[0],"strand":introns[5]}
introndata_df = pd.DataFrame(introndata,columns=['chrom','start','end','refseq_id','strand'])
introndata_df_validchrm = introndata_df[introndata_df["chrom"].isin(chrom_act)]
print introndata_df_validchrm.shape
introndata_df_validchrm_NM = introndata_df_validchrm[introndata_df_validchrm['refseq_id'].str.contains('NM')]
print introndata_df_validchrm_NM.shape
introndata_df_validchrm_NM.head()

(476743, 5)
(424240, 5)


,chrom,start,end,refseq_id,strand
9,chr1,67093604,67096251,NM_001276352,-
10,chr1,67096321,67103237,NM_001276352,-
11,chr1,67103382,67111576,NM_001276352,-
12,chr1,67111644,67115351,NM_001276352,-
13,chr1,67115464,67125751,NM_001276352,-


In [14]:
# We want to get the RefSeq ids found in the NCBI curated intron data 
refseqids_introndata = introndata_df_validchrm_NM["refseq_id"].drop_duplicates()
print refseqids_introndata.shape
refseqids_introndata.head()

(41707,)


9     NM_001276352
17    NM_001276351
24    NM_001005337
37       NM_000299
51    NM_001042682
Name: refseq_id, dtype: object

In [10]:
# Grab the mapping file that maps refseq IDs to gene names
# Only grab the IDs that are for mRNAs
refseqid_gene = pd.read_csv('../data/Refseq2Gene.txt',sep='\t',header=None)
refseqid_gene_nodups = refseqid_gene.drop_duplicates()
refseqid_gene_nodups.columns = ['refseq_id_with_gene','gene_name']
refseqid_gene_nodups_NM = refseqid_gene_nodups[refseqid_gene_nodups['refseq_id_with_gene'].str.contains('NM')]
print refseqid_gene_nodups_NM.shape
refseqid_gene_nodups_NM.head()

(50105, 2)


,refseq_id_with_gene,gene_name
1,NM_015537,NSMF
27,NM_033013,NR1I2
28,NM_033012,TNFSF11
29,NM_032898,CEP19
30,NM_033007,NLRP1


In [15]:
refseq_IDs_not_available_for_introns = [i for i in list(refseqids_introndata) if i not in list(refseqid_gene_nodups_NM["refseq_id_with_gene"].values)] 
len(refseq_IDs_not_available_for_introns)
# I checked these refseqIDs that were missing in the mapping of refseqID to gene 
# but were present in the intron list on NCBI. Those refseq IDs record has been removed or replaced. 
# So we should get rid of them

84

In [18]:
# Refseq IDs present in both intron data and mapping data
refseq_IDs_available_for_introns = [i for i in list(refseqids_introndata) if i in list(refseqid_gene_nodups_NM["refseq_id_with_gene"].values)] 

In [21]:
# Only get gene names for which there are valid refseq ids in the intron data
refseqid_gene_nodups_NM_ValidRefSeqIDs = refseqid_gene_nodups_NM[refseqid_gene_nodups_NM["refseq_id_with_gene"].isin(refseq_IDs_available_for_introns)]
print refseqid_gene_nodups_NM_ValidRefSeqIDs.shape

(41623, 2)


In [22]:
# Only get introns that have valid gene names in mapping data
introndata_df_validchrm_NM_ValidRefSeqIDs = introndata_df_validchrm_NM[introndata_df_validchrm_NM['refseq_id'].isin(refseq_IDs_available_for_introns)]
print introndata_df_validchrm_NM_ValidRefSeqIDs.shape

(423486, 5)


Merge the intron data which is referenced by NCBI RefSeqID to genename-RefSeqID data so that we can associate introns to a gene name

In [24]:
introndata_withgenename = introndata_df_validchrm_NM_ValidRefSeqIDs.merge(refseqid_gene_nodups_NM_ValidRefSeqIDs,left_on="refseq_id",right_on="refseq_id_with_gene")
introndata_withgenename = introndata_withgenename[['chrom','start','end','refseq_id','strand','gene_name']]
print introndata_withgenename.shape
introndata_withgenename.head()

(423486, 6)


,chrom,start,end,refseq_id,strand,gene_name
0,chr1,67093604,67096251,NM_001276352,-,C1orf141
1,chr1,67096321,67103237,NM_001276352,-,C1orf141
2,chr1,67103382,67111576,NM_001276352,-,C1orf141
3,chr1,67111644,67115351,NM_001276352,-,C1orf141
4,chr1,67115464,67125751,NM_001276352,-,C1orf141


In [25]:
# Write this dataset into a file which will be used to intersect with mutation data
introndata_withgenename.to_csv("../processed_data/Introns_JustmRNAs.bed",sep="\t",header=False,index=False)

We are now going to categorize the introns based on if there are alterative or constitutive introns. 

Alternative introns are sometimes spliced in or not depending on isoform for a given gene, 

while constitutive introns are always spliced out in all known isoforms of a gene. 

In [26]:
# Each RefSeq ID represents an isoform. Count the number of isoforms (number of RefSeq IDs) attached to a gene 
# and merge that count with previous dataset  
num_refseqids_per_gene = pd.DataFrame({'count' : refseqid_gene_nodups_NM_ValidRefSeqIDs.groupby( [ "gene_name"] ).size()}).reset_index()
introndata_withgenename_countdata = introndata_withgenename.merge(num_refseqids_per_gene,left_on="gene_name",right_on="gene_name")
print introndata_withgenename_countdata.shape
introndata_withgenename_countdata.head()

(423486, 7)


,chrom,start,end,refseq_id,strand,gene_name,count
0,chr1,67093604,67096251,NM_001276352,-,C1orf141,2
1,chr1,67096321,67103237,NM_001276352,-,C1orf141,2
2,chr1,67103382,67111576,NM_001276352,-,C1orf141,2
3,chr1,67111644,67115351,NM_001276352,-,C1orf141,2
4,chr1,67115464,67125751,NM_001276352,-,C1orf141,2


In [31]:
# Group all the data by gene_name, chrom, start and end coordinates, and strand and then expand all the grouped data. 
# The purpose of doing this is so that we can get unique coordinates for the introns of each gene. 
introns_groupedByGenes_IntronPosition = introndata_withgenename_countdata.groupby(['gene_name','count','chrom','start','end','strand']).count()
introns_groupedByGenes_IntronPosition.reset_index(inplace=True)
print introns_groupedByGenes_IntronPosition.shape
introns_groupedByGenes_IntronPosition.head()

(209014, 7)


,gene_name,count,chrom,start,end,strand,refseq_id
0,A1BG,1,chr19,58347029,58347352,-,1
1,A1BG,1,chr19,58347640,58350369,-,1
2,A1BG,1,chr19,58350651,58351390,-,1
3,A1BG,1,chr19,58351687,58352282,-,1
4,A1BG,1,chr19,58352555,58352927,-,1


In [28]:
# We can now separate between constitutive introns and alternative introns 
# since constitutive introns will have the same count as the number of refseqids 
constitutive_introns = introns_groupedByGenes_IntronPosition[introns_groupedByGenes_IntronPosition['count']==introns_groupedByGenes_IntronPosition['refseq_id']]
constitutive_introns_sorted = constitutive_introns[["chrom","start","end","gene_name","strand"]].sort_values(["chrom","start"])
constitutive_introns_sorted.to_csv("../processed_data/constitutive_introns.bed",sep="\t",header=False,index=False)
print constitutive_introns.shape
constitutive_introns_sorted.head()

(149474, 7)


,chrom,start,end,gene_name,strand
156785,chr1,925800,925921,SAMD11,+
156786,chr1,926013,930154,SAMD11,+
156787,chr1,930336,931038,SAMD11,+
156788,chr1,931089,935771,SAMD11,+
156789,chr1,935896,939039,SAMD11,+


In [29]:
# alternative introns will not have the same count sa number of refseqids
alternative_introns = introns_groupedByGenes_IntronPosition[introns_groupedByGenes_IntronPosition['count']!=introns_groupedByGenes_IntronPosition['refseq_id']]
alternative_introns_sorted = alternative_introns[["chrom","start","end","gene_name","strand"]].sort_values(["chrom","start"])
alternative_introns_sorted.to_csv("../processed_data/alternative_introns.bed",sep="\t",header=False,index=False)
print alternative_introns.shape
alternative_introns_sorted.head()

(59540, 7)


,chrom,start,end,gene_name,strand
136626,chr1,971006,971076,PLEKHN1,+
136627,chr1,971006,971112,PLEKHN1,+
136632,chr1,973010,973185,PLEKHN1,+
136633,chr1,973010,973499,PLEKHN1,+
136634,chr1,973326,973499,PLEKHN1,+


In [2]:
%%script bash
# We want introns that are not intersecting with any exons: They are pure introns -> 
# this way any mutations found in those introns are not affecting coding parts
bedtools intersect -a ../data/NCBI_RefSeq_Curated_Introns.bed -b ../data/NCBI_RefSeq_Curated_Exons.bed -wa -v > ../processed_data/Introns_DoNotIntersect_Exons.bed